
# Quantum Computing Mini‑Course (Qiskit, Hands‑On)

Welcome! This notebook is a guided, **learn-by-doing** introduction to quantum mechanics concepts as they appear in **quantum computing**.

Each lesson has:
- *Concept* – short, plain‑language explanation  
- *Principle* – minimal math intuition  
- *Code* – runnable Qiskit cells  
- *Exercises* – things for you to tweak

> **Tip:** Run cells in order. If you haven't installed Qiskit yet, run the next cell first.


In [ ]:
# (Optional) Install or update the required packages
# If you're running locally and haven't installed these yet, uncomment and run:
# !pip install -U "qiskit>=1.0" "qiskit-aer>=0.15" "qiskit-ibm-runtime>=0.25"


In [1]:
# Core imports (Qiskit 1.x compatible API)
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit.quantum_info import Statevector
import matplotlib.pyplot as plt

# We'll use a single shot-based simulator for measurement experiments
qasm_sim = AerSimulator()

# Make plots a bit larger
plt.rcParams["figure.figsize"] = (6, 4)
print("Imports ready. If no errors appeared, you're good to go!")

Imports ready. If no errors appeared, you're good to go!



---
## Lesson 1 — Superposition

📖 **Concept**  
A qubit can be in |0⟩, |1⟩, or any mix of the two. The Hadamard gate (H) puts a qubit into equal superposition.

🧮 **Principle**  
\[ H|0\rangle = \tfrac{|0\rangle + |1\rangle}{\sqrt{2}} \]


In [2]:
# Hadamard on |0>, observe the statevector and Bloch sphere
qc = QuantumCircuit(1)
qc.h(0)

psi = Statevector.from_instruction(qc)
print("Statevector:", psi.data)

plot_bloch_multivector(psi)
plt.show()

Statevector: [0.70710678+0.j 0.70710678+0.j]



🎯 **Exercises**
- Replace `qc.h(0)` with `qc.x(0)`. Where does the state move on the Bloch sphere?
- Try `qc.h(0); qc.x(0)` or `qc.rx(theta, 0)` with different angles.



---
## Lesson 2 — Measurement

📖 **Concept**  
Measurement collapses the state to a definite 0 or 1. Superpositions show up as **probabilities**.

🧮 **Principle**  
Measuring \( \tfrac{|0\rangle + |1\rangle}{\sqrt{2}} \) yields ~50% 0 and ~50% 1.


In [3]:
# Create a superposition and measure many times (shots)
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.measure(0, 0)

compiled = transpile(qc, qasm_sim)
result = qasm_sim.run(compiled, shots=2000).result()
counts = result.get_counts()

print("Counts:", counts)
plot_histogram(counts)
plt.show()

Counts: {'1': 1047, '0': 953}



🎯 **Exercises**
- Increase `shots` (e.g., 10_000). Do the frequencies get closer to 50/50?
- Replace `qc.h(0)` with `qc.x(0)`. What histogram do you get and why?



---
## Lesson 3 — Entanglement

📖 **Concept**  
Two qubits can be correlated beyond classical limits. In the Bell state, measuring one tells you the other instantly.

🧮 **Principle**  
\[ |\Phi^+\rangle = \tfrac{|00\rangle + |11\rangle}{\sqrt{2}} \]


In [4]:
# Create a Bell state and measure both qubits
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

compiled = transpile(qc, qasm_sim)
result = qasm_sim.run(compiled, shots=4000).result()
counts = result.get_counts()

print("Counts:", counts)
plot_histogram(counts)
plt.show()

Counts: {'00': 2002, '11': 1998}



🎯 **Exercises**
- Do you see only `00` and `11`? (Small noise on real hardware may add others.)
- Add `qc.h(1)` after `qc.cx(0, 1)`. What changes?
- Try `qc.z(0)` before `qc.cx(0, 1)`. How does a phase affect outcomes?



---
## Lesson 4 — Interference

📖 **Concept**  
Quantum states act like waves: phases can add (constructive) or cancel (destructive).

🧮 **Principle**  
Two Hadamards undo each other: \( H(H|0\rangle) = |0\rangle \).


In [ ]:
# Interference: H; H; measure -> always 0
qc = QuantumCircuit(1, 1)
qc.h(0)
qc.h(0)
qc.measure(0, 0)

compiled = transpile(qc, qasm_sim)
result = qasm_sim.run(compiled, shots=2000).result()
counts = result.get_counts()

print("Counts:", counts)
plot_histogram(counts)
plt.show()


🎯 **Exercise**
- Insert a phase between the Hadamards: `qc = QuantumCircuit(1,1); qc.h(0); qc.z(0); qc.h(0); ...`
- Predict the histogram before you run it.



---
## Lesson 5 — First Algorithm: Deutsch–Jozsa (Lite)

📖 **Concept**  
Deutsch–Jozsa distinguishes between a *constant* vs. *balanced* oracle in a **single** evaluation by using interference.

For a 1‑qubit "toy" oracle:
- Constant oracle ≈ Identity (I)  → outputs 0
- Balanced oracle ≈ Phase flip (Z) → outputs 1


In [ ]:
# "Toy" Deutsch–Jozsa: single-qubit version for intuition
def deutsch_jozsa_lite(use_balanced_oracle: bool = True, shots: int = 1000):
    qc = QuantumCircuit(1, 1)
    qc.h(0)                 # prepare superposition
    
    if use_balanced_oracle:
        qc.z(0)             # balanced oracle (phase flip)
    else:
        qc.i(0)             # constant oracle (does nothing)
    
    qc.h(0)
    qc.measure(0, 0)
    
    compiled = transpile(qc, qasm_sim)
    result = qasm_sim.run(compiled, shots=shots).result()
    return result.get_counts()

print("Balanced oracle (expect '1'):", deutsch_jozsa_lite(True))
print("Constant oracle (expect '0'):", deutsch_jozsa_lite(False))


🎯 **Exercises**
- Vary `shots` and confirm the dominant outcome remains the same.
- Replace `qc.z(0)` with `qc.rx(theta, 0)` for various `theta`. When does it look "balanced"?



---
## 🚀 Bonus — Try a Real Quantum Device (IBM)

1. Create a free IBM Quantum account and save your token:
   - Visit https://quantum-computing.ibm.com/
   - Follow instructions to store your account credentials.

2. Replace the simulator with a real backend:


In [ ]:
# Example sketch for real hardware (commented out to avoid errors if not configured).
# from qiskit_ibm_runtime import QiskitRuntimeService
#
# service = QiskitRuntimeService()  # assumes you configured your IBM credentials
# backend = service.least_busy(filters=lambda b: not b.simulator and b.num_qubits >= 2)
#
# # Reuse a circuit from above, e.g., the Bell state:
# qc = QuantumCircuit(2, 2)
# qc.h(0)
# qc.cx(0, 1)
# qc.measure([0, 1], [0, 1])
#
# compiled = transpile(qc, backend)
# job = backend.run(compiled, shots=4000)
# result = job.result()
# print(result.get_counts())


---

### Where to go next
- Implement **Grover's search** for 2 qubits (mark one state with a phase oracle).
- Implement **Quantum teleportation** to deepen your understanding of entanglement.
- Explore **noise** by adding error models in `qiskit-aer` or by comparing simulator vs. hardware results.

Happy hacking ✨
